In [1]:
import regex as re
import pandas as pd
import numpy as np
import itertools

In [2]:
excel_file = pd.ExcelFile(r'H:\PythonProjects\ABB_EIO_translation\files\EIO_test1.xlsx')

In [3]:
df_input = pd.read_excel(excel_file, sheet_name='INPUT')
df_output = pd.read_excel(excel_file, sheet_name='OUTPUT')

In [4]:
df_all = df_input.append(df_output)

In [5]:
df_all.set_index('Name') # jesli unikatowe
df_all = df_all.drop(['Unnamed: 0', 'DeviceMapToSort'], axis = 1)

In [6]:
df_all = df_all.reset_index()


In [7]:
columns_name = ['Name', 'SignalType', 'Device', 'Label', 'DeviceMap', 'Category', 'Access', 'Default', 'SafeLevel', 'EncType', 'MaxLog', 'MaxPhys', 'MaxPhysLimit', 'MaxBitVal', 'MinLog', 'MinPhys', 'MinPhysLimit', 'MinBitVal']
with_apostrophe = ['Name', 'SignalType', 'Device', 'Label', 'DeviceMap', 'Category', 'Access', 'SafeLevel', 'EncType']
without_apostrophe = set(columns_name).difference(set(with_apostrophe))

In [8]:
def write_line_to_cfg(line):
    result_string = ''
    for column in columns_name:
        if not pd.isnull(line[column]):
            if column in with_apostrophe:
                try: 
                    if line[column].upper() not in ['NAN']:
                        result_string += f'-{column} "{line[column]}"\\\n'
                except:
                    pass
            if column in without_apostrophe:
                 result_string += f'-{column}{line[column]}\\\n'
    return result_string[:-2] + '\n'*2
    

In [9]:
to_print = df_all.apply(write_line_to_cfg, axis=1)

In [10]:
with open('result_EIO.txt', 'w') as f:
    #to_print = df_all['result_str'].to_string(index=False)
    f.writelines(to_print)

In [11]:
with open('result_EIO.txt', 'r') as f:
    #to_print = df_all['result_str'].to_string(index=False)
    strin = f.read()

In [12]:
df_all[['Label', 'Category', 'Access', 'SafeLevel', 'EncType']] = df_all[['Label', 'Category', 'Access', 'SafeLevel', 'EncType']].astype('str')
def upper_case_nan(x):
    return 'NAN' if x == 'nan' else x
df_all['Label'] = df_all['Label'].apply(upper_case_nan)
df_all['Category'] = df_all['Category'].apply(upper_case_nan)
df_all['Access'] = df_all['Access'].apply(upper_case_nan)
df_all['SafeLevel'] = df_all['SafeLevel'].apply(upper_case_nan)
df_all['EncType'] = df_all['EncType'].apply(upper_case_nan)

In [13]:
# df_all.loc[1, 'SignalType'] = 'da'
# df_all.loc[0, 'Label'] = '@#'
df_all.loc[1, 'Name'] = 'dsd$'
#df_all.loc[0, 'DeviceMap'] = '111111'
# df_all.loc[1, 'Category'] = 'asdas&'
# df_all.loc[1, 'Access'] = 'asd'
# df_all.loc[1, 'SafeLevel'] = 'aasdasdsa'
#df_all.loc[1, 'EncType'] = 'asdas'
#df_all.loc[1, 'MaxLog'] = 'asdas'
#df_all.loc[1, 'MaxPhysLimit'] = 'asdas'
#df_all.loc[1, 'Default'] = 'asdas'

In [14]:
available_signal_type = ['AI', 'AO', 'DI', 'DO', 'GI', 'GO']
def check_signal_type(line):
    if line['SignalType'].upper() in available_signal_type:
        line['SignalType'] = line['SignalType'].upper()
        return line
    line['SignalType'] = input(f'Wrong type of signal {line.Name}. Current type: {line.SignalType}  \n Enter correct type: ')
    return check_signal_type(line)
    

In [15]:
def check_correct_character(string_to_check, regex_str = r'[_a-zA-Z0-9]+', available_null=False):
    if not available_null: #dodac nulla
        pattern = re.compile(regex_str)
        return pattern.fullmatch(string_to_check)


In [16]:
def check_name(line, column_name):
    print(type(line))
    if check_correct_character(line[column_name]): 
        return line
    line[column_name] = input(f'Wrong {column_name}: {line[column_name]}. \nEnter correct {column_name}: ')
    return check_name(line, column_name)


In [17]:
def check_device_str(name, column_name):
    if check_correct_character(name): 
        return name
    name = input(f'Wrong {column_name} name: {name}. \nEnter correct name: ')
    return check_device_str(name, column_name)

In [18]:
def check_label_str(label, column_name='Label'):
    if check_correct_character(label, regex_str='[\w _]+'): 
        return label
    label = input(f'Wrong {column_name} name: {label}. \nEnter correct name: ')
    return check_label_str(label, column_name)


In [19]:
def check_mapping(line, column_name='DeviceMap'):
    pattern = re.compile(r'(\d{1,4})(-\d{1,4})?')
    if pattern.fullmatch(line['DeviceMap'].strip()):
        return line
    line['DeviceMap'] = input(f'Wrong {column_name} name: {line[column_name]}. \nEnter correct {column_name}: ')
    return check_mapping(line, column_name)

In [20]:
def check_category(line, column_name='Category'):
    if check_correct_character(line['Category']): 
        return line
    line['Category'] = input(f'Wrong {column_name} name: {line.Category}. \nEnter correct name: ')
    return check_category(line, column_name)


In [21]:
available_signal_safe_level = ['SAFETYSAFELEVEL', 'NAN']
def check_safe_level(line, column_name='safe_level'):
    if line['SafeLevel'].strip().upper() in available_signal_safe_level:
        return line
    line['SafeLevel'] = input(f'Wrong {column_name}. Current type: {line.SafeLevel}  \n Enter correct type: ')
    return check_safe_level(line)

In [22]:
available_signal_access = ['READONLY', 'NAN']
def check_access(line, column_name='Access'):
    if line['Access'].strip().upper() in available_signal_access:
        return line
    line['Access'] = input(f'Wrong {column_name}. Current type: {line.Access}  \n Enter correct type: ')
    return check_access(line)

In [23]:
available_signal_enctype = ['UNSIGNED', 'SIGNED', 'NAN']
def check_enctype(line, column_name='EncType'):
    if line['EncType'].strip().upper() in available_signal_enctype:
        line['EncType'] = line['EncType'].upper()
        return line
    line['EncType'] = input(f'Wrong {column_name}.{type(line.EncType)}  Current type: {line.EncType}  \n Enter correct type: ')
    return check_enctype(line)

In [24]:
def check_is_digit(line, column_name=[]):
    for column in column_name:
        try:
            line[column_name] = float(line[column])
        except:
            line[column] = input(f'{column} is not digit. Current value: {line[column]}  \n Enter correct value: ')
            check_is_digit(line, [column])
    line[column_name] = line[column_name].astype('float')
    return line

In [25]:
def try_read_number(text):
    try:
        return float(input(text))
        
    except:
        print('To nie jest liczba')

In [26]:
def check_default(line, column_name='Default'):
    try:
        to_test = float(line[column_name])
        if line['SignalType'] in ['DI', 'DO'] and not line[column_name] in [0, 1] \
        or line['SignalType'] in ['GI', 'GO'] and not line[column_name] % 1 == 0:
            if not pd.isna(line[column_name]): 
                raise 'make exception'
        return line
    except:
        line[column_name] = try_read_number(f'Default value in signal {line.Name} is not valid. Current value: {line.Default} in signal type {line.SignalType} \n Enter correct value: ')
        check_default(line)
    return line

In [28]:
def check_all_cells(line):
    line['Name'] = check_name(line, 'Name')
    line['Device'] = check_device_str(line['Device'], 'Device')
    line['Label'] = check_label_str(line['Label'])
    line = check_signal_type(line)
    line = check_mapping(line)
    line = check_category(line)
    line = check_safe_level(line)
    line = check_access(line)
    line = check_enctype(line)
    line = check_default(line)
    # dodać podział na int i float?
    line = check_is_digit(line, ['MaxLog', 'MaxPhys', 'MaxPhysLimit', 'MaxBitVal', 'MinLog', 'MinPhys', 'MinPhysLimit', 'MinBitVal']) 
    return line
    

In [ ]:
df_all = df_all.apply(check_all_cells, 1)

<class 'pandas.core.series.Series'>
Wrong DeviceMap name: 111111. 
Enter correct DeviceMap: 12
<class 'pandas.core.series.Series'>


In [ ]:
df_all.head()

In [ ]:
df_all['Default'].describe()

In [43]:
int(df_all.loc[2, 'Default'])

2

In [ ]:
if df_all.loc[2, 'Default'] is None: 
    print('jestem tu')
else:
    print('zle')

In [50]:
isinstance(True, int)

True

In [48]:
type(int(df_all.loc[2, 'Default']))

int

In [ ]:
pd.isna


In [ ]:
pd.isna(df_all.loc[2,'Default'])


In [35]:
df_all


,index,Name,SignalType,Device,Label,DeviceMap,Category,Access,Default,SafeLevel,EncType,MaxLog,MaxPhys,MaxPhysLimit,MaxBitVal,MinLog,MinPhys,MinPhysLimit,MinBitVal
0,0,DI_PP_To_Main,DI,PN_Internal_Device,SI pp_tp_main,0,EIOtest,ReadOnly,1.0,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dsd$,DI,PN_Internal_Device,SI Stop robot,1,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,DI_Start_Robot,DI,PN_Internal_Device,SI Start robot,2,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,DI_Motor_On_Robot,DI,PN_Internal_Device,SI Motor on,3,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,DI_Motor_Off_Robot,DI,PN_Internal_Device,SI Motor off request,4,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,DI_Reset_Robot_Error,DI,PN_Internal_Device,SI Reset error,5,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,DI_Pgno_Valid,DI,PN_Internal_Device,Valid program nr,16,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,DI_Work_HMI,DI,PN_Internal_Device,Work by HMI,17,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,DI_DryCycle,DI,PN_Internal_Device,Dry cycle,18,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,GI_Pgno_Nr,GI,PN_Internal_Device,Program nr,32-39,NAN,NAN,NaN,NAN,NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
column_name = 'DeviceMap'
temp = df_all.loc[0] 
temp[column_name]= input(f'Wrong {column_name} name: {temp[column_name]}. \nEnter correct {column_name}: ')

In [ ]:
temp


In [55]:
test_str = 'moj str '
test_str
test_str.strip()

'moj str'

In [56]:
df_all['SafeLevel']

0                 NAN
1                 NAN
2                 NAN
3                 NAN
4                 NAN
5                 NAN
6                 NAN
7                 NAN
8                 NAN
9                 NAN
10                NAN
11                NAN
12                NAN
13                NAN
14                NAN
15                NAN
16                NAN
17                NAN
18                NAN
19                NAN
20                NAN
21                NAN
22                NAN
23                NAN
24                NAN
25                NAN
26                NAN
27                NAN
28                NAN
29                NAN
30                NAN
31                NAN
32                NAN
33                NAN
34                NAN
35                NAN
36                NAN
37                NAN
38                NAN
39                NAN
40                NAN
41                NAN
42                NAN
43                NAN
44    SafetySafeLevel
45    Safe

In [3]:
a = np.nan

In [16]:
df_test = pd.DataFrame([a, a], columns = ['test'])

In [23]:
df_test = df_test.astype('str')

In [26]:
df_test['test'].str.upper()

0    NAN
1    NAN
Name: test, dtype: object